In [1]:
import os
_origlistdir = os.listdir

In [2]:
permute = True
load_features = False
target_subject = 'p024'
save_features = !load_features
save_weights = True

In [3]:
%%bash
split_name='sample'

rm -rf splits/$split_name
mkdir -p splits/$split_name/train
for obj_type in `ls train`; do
  mkdir -p splits/$split_name/train/$obj_type
  mkdir -p splits/$split_name/valid/$obj_type

  #echo "Preparing training set for class ${obj_type}"
  ls train/$obj_type | tail -n150 | xargs -I{} cp train/$obj_type/{} splits/$split_name/train/$obj_type
  #echo "Preparing validation set for class ${obj_type}"
  ls train/$obj_type | tail -n200 | head -n50 | xargs -I{} cp train/$obj_type/{} splits/$split_name/valid/$obj_type
done

In [4]:
from keras.preprocessing.image import ImageDataGenerator
batch_size = 32

def setup_filter_to_exclude(notallowed_filenames):
    def listdir(path):
        res = _origlistdir(path)
        res = list(filter(lambda fname: fname not in notallowed_filenames, res))
        return res
    os.listdir = listdir

def clear_filter():
    os.listdir = _origlistdir

def data_for(validation_subject_id, shuffle=False, verbose=False):
    generator = ImageDataGenerator()

    train_fnames = set(drivers[drivers['subject'] != validation_subject_id]['img'])
    validation_fnames = set(drivers[drivers['subject'] == validation_subject_id]['img'])

    # train
    setup_filter_to_exclude(validation_fnames)
    train_generator = generator.flow_from_directory(
        'train',
#        'splits/sample/train',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=shuffle)

    # validation
    setup_filter_to_exclude(train_fnames)
    validation_generator = generator.flow_from_directory(
#        'splits/sample/valid',
        'train',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=shuffle)

    clear_filter()

    return train_generator, validation_generator

Using Theano backend.
WARNING (theano.gof.compilelock): Overriding existing lock by dead process '14011' (I am process '1380')
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [38]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img, img_to_array
from utils import *

def get_conv_layers():
    from keras.applications.vgg16 import VGG16
    vgg16 = VGG16()
    conv_layers, fc_layers = split_at(vgg16, Convolution2D)
    return conv_layers

def get_bn_layers_with_dropoff(p):
    from keras.applications.vgg16 import VGG16
    vgg16 = VGG16()
    conv_layers, fc_layers = split_at(vgg16, Convolution2D)
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        BatchNormalization(axis=1, mode=2),
        Dropout(p),
        Dense(4096, activation='relu'),
        BatchNormalization(axis=1, mode=2),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

def proc_wgts(layer): return [o/2 for o in layer.get_weights()]

def get_fc_model(conv_layers, fc_layers):
    fc_model = Sequential([
    MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
    Flatten(),
    Dense(4096, activation='relu'),
    Dense(4096, activation='relu'),
    Dense(10, activation='softmax')
    ])

    for l1,l2 in zip(fc_model.layers, fc_layers)[:-1]: l1.set_weights(proc_wgts(l2))

    from keras.optimizers import RMSprop
    opt = RMSprop(lr=0.00001, rho=0.7)
    fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return fc_model

In [407]:
def rescale_dense(bn):
    def proc_wgts(layer, prev_p, new_p):
        scal = (1-prev_p)/(1-new_p)
        return [o*scal for o in layer.get_weights()]

    for l in bn.layers:
        if type(l)==Dense: l.set_weights(proc_wgts(l, 0.5, 0.6))

#rescale_dense(bn)

In [39]:
conv_layers = get_conv_layers()
conv_model = Sequential(conv_layers)

In [40]:
import pandas as pd
drivers = pd.DataFrame.from_csv('driver_imgs_list.csv', index_col=None)
train_batches, validation_batches = data_for(target_subject)
from keras.utils.np_utils import to_categorical
train_labels = to_categorical(train_batches.classes)
validation_labels = to_categorical(validation_batches.classes)

if (len(np.unique(validation_batches.classes)) != len(np.unique(train_batches.classes))):
    raise Exception("Not all the classes are represented in the sample")

model_path='/home/ubuntu/downloads/'
if (load_features):
    print("Loading outputs of conv model")
    train_features      = load_array(model_path + 'train_convlayer_features.bc')
    validation_features = load_array(model_path + 'valid_convlayer_features.bc')
    train_labels        = load_array(model_path + 'train_convlayer_labels.bc')
    validation_labels   = load_array(model_path + 'valid_convlayer_labels.bc')
    print("Done loading outputs of conv model")
if not load_features:
    print("Calculating outputs of conv_model")
    validation_features = conv_model.predict_generator(validation_batches, validation_batches.nb_sample)
    train_features = conv_model.predict_generator(train_batches, train_batches.nb_sample)
    print("Done calculating outputs of conv_model")
# shuffle
if (permute):
    print("Permuting training data")
    tr_permutation = np.random.permutation(len(train_features))
    train_features = train_features[tr_permutation]
    train_labels   = train_labels[tr_permutation]
    v_permutation       = np.random.permutation(len(validation_features))
    validation_features = validation_features[v_permutation]
    validation_labels   = validation_labels[v_permutation]
    print("Permuting training data")

Found 21198 images belonging to 10 classes.
Found 1226 images belonging to 10 classes.
Calculating outputs of conv_model
Done calculating outputs of conv_model
Permuting training data
Permuting training data


In [41]:
bn = Sequential(get_bn_layers_with_dropoff(0.4))

def load_fc_weights_from_vgg16bn(model):
    from vgg16bn import Vgg16BN
    vgg16bn = Vgg16BN()
    _, fc_layeres = split_at(vgg16bn.model, Convolution2D)
    for l1,l2 in zip(model.layers, fc_layers)[:-1]: l1.set_weights(proc_wgts(l2))
    
load_fc_weights_from_vgg16bn(bn)
bn.pop()
bn.add(Dense(10, activation='softmax'))

In [42]:
from keras.optimizers import Adam
for layer in bn.layers[:-4]: layer.trainable = False
bn.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])
bn.optimizer.lr=0.001

In [43]:
train_features.shape, train_labels.shape, validation_features.shape, validation_labels.shape

((21198, 512, 14, 14), (21198, 10), (1226, 512, 14, 14), (1226, 10))

In [44]:
[layer.trainable for layer in bn.layers]

[False, False, False, False, False, True, True, True, True]

In [45]:
bn.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
maxpooling2d_34 (MaxPooling2D)   (None, 512, 7, 7)     0           maxpooling2d_input_4[0][0]       
____________________________________________________________________________________________________
flatten_10 (Flatten)             (None, 25088)         0           maxpooling2d_34[0][0]            
____________________________________________________________________________________________________
dense_29 (Dense)                 (None, 4096)          0           flatten_10[0][0]                 
____________________________________________________________________________________________________
batchnormalization_19 (BatchNorma(None, 4096)          0           dense_29[0][0]                   
___________________________________________________________________________________________

In [46]:
bn.fit(train_features, train_labels, nb_epoch=2, validation_data=(validation_features, validation_labels), verbose=1)

Train on 21198 samples, validate on 1226 samples
Epoch 1/2
21198/21198 [==============================] - 24s - loss: 0.5492 - acc: 0.8886 - val_loss: 2.6872 - val_acc: 0.6656
Epoch 2/2
21198/21198 [==============================] - 24s - loss: 0.1805 - acc: 0.9556 - val_loss: 3.1285 - val_acc: 0.6199


In [88]:
%%notify
print("Done")

ERROR:root:Cell magic `%%notify` not found.


In [290]:
if (save_features):
    print("Saving outputs of conv model")
    model_path='/home/ubuntu/downloads/'
    save_array(model_path + 'train_convlayer_features.bc', train_features)
    save_array(model_path + 'valid_convlayer_features.bc', validation_features)
    save_array(model_path + 'train_convlayer_labels.bc', train_labels)
    save_array(model_path + 'valid_convlayer_labels.bc', validation_labels)
    print("Done saving outputs of conv model")

Saving outputs of conv model
Done saving outputs of conv model


In [324]:
if (save_weights):
    bn.save_weights(model_path+'bn_fc_5_epoch.h5')
    print("Done saving weights of fc bn model")

Done saving weights of fc bn model


In [32]:
final_model = Sequential(conv_layers)
for layer in final_model.layers: layer.trainable = False

In [33]:
for layer in bn.layers: final_model.add(layer)

In [34]:
final_model.compile(optimizer=Adam(), 
                    loss='categorical_crossentropy', metrics=['accuracy'])
final_model.optimizer.lr=0.1

In [35]:
last_conv_idx = [idx for idx,layer in enumerate(final_model.layers) if type(layer) is Convolution2D][-1]
for layer in final_model.layers[last_conv_idx:]: layer.trainable = True

In [36]:
final_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 3, 224, 224)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 64, 224, 224)  0           input_4[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 64, 224, 224)  0           block1_conv1[2][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 64, 112, 112)  0           block1_conv2[2][0]               
___________________________________________________________________________________________

In [37]:
final_model.optimizer.lr=0.001
final_model.fit_generator(train_batches, samples_per_epoch=train_batches.nb_sample, nb_epoch=5, 
                        validation_data=validation_batches, nb_val_samples=validation_batches.nb_sample)

Epoch 1/5
21198/21198 [==============================] - 599s - loss: 11.1763 - acc: 0.2922 - val_loss: 13.4076 - val_acc: 0.1020
Epoch 2/5
21198/21198 [==============================] - 601s - loss: 14.1311 - acc: 0.1187 - val_loss: 14.1599 - val_acc: 0.0865
Epoch 3/5
 3360/21198 [===>..........................] - ETA: 479s - loss: 13.1293 - acc: 0.1801

KeyboardInterrupt: 

In [76]:
final_model.save_weights(model_path + 'final2.h5')

In [77]:
import pandas as pd
from IPython.display import FileLink
def predict_to_csv(model, filename='submission.csv'):
    generator = ImageDataGenerator()    
    test_generator = generator.flow_from_directory(
        'test',
        target_size=(224, 224),
        batch_size=batch_size,
        shuffle=False,
        class_mode=None)
    
    predictions = model.predict_generator(test_generator, test_generator.nb_sample)
    filenames = map(lambda name: name.split('/')[1], test_generator.filenames)

    df = pd.DataFrame(predictions,index=filenames)
    df.columns = map(lambda (a,b): b + str(a), zip(range(0,10), ['c'] * 10))

    df.to_csv(filename, index_label='img')

    return FileLink(filename)

In [78]:
# 31.76 = doesn't look good at all
predict_to_csv(final_model, 'submission_batchnorm.csv')

Found 79726 images belonging to 1 classes.


/home/ubuntu/downloads/submission_batchnorm.csv

In [80]:
train_batches_shuffled, validation_batches_shuffled = data_for('p002', shuffle=True)

Found 21699 images belonging to 10 classes.
Found 725 images belonging to 10 classes.


In [83]:
final_model.optimizer.lr=0.000001
final_model.fit_generator(train_batches_shuffled, samples_per_epoch=train_batches_shuffled.nb_sample, nb_epoch=1, 
                        validation_data=validation_batches_shuffled, nb_val_samples=validation_batches_shuffled.nb_sample)

Epoch 1/1
  480/21699 [..............................] - ETA: 672s - loss: 14.9092 - acc: 0.0750

KeyboardInterrupt: 

In [92]:
%ls splits/sample

train/  valid/
